In [1]:
''' 1. Module Import '''
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.9.0+cpu  Device: cpu


In [3]:
# 배치와 에폭 설정
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
''' 3. MNIST 데이터 다운로드 (Train set, Test set 분리하기) '''
train_dataset = datasets.MNIST(root = "../data/MNIST",
                               train = True,          # train용
                               download = True,
                               transform = transforms.ToTensor())

test_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = False,          # test용 
                              transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

C:\Users\tjdqj\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
''' 4. 데이터 확인하기 (1) '''
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break
    
    # 32의 배치사이즈, 1개의 데이터, 28, 28 크기의 데이터
    # 레이블은 32개의 데이터

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [6]:
# ''' 5. 데이터 확인하기 (2) '''
# pltsize = 1
# plt.figure(figsize=(10 * pltsize, pltsize))
# for i in range(10):
#     plt.subplot(1, 10, i + 1)
#     plt.axis('off')
#     plt.imshow(X_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
#     plt.title('Class: ' + str(y_train[i].item()))

## 로컬에서 실행 시 커널이 죽어버림.

In [7]:
''' 6. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512) # fully connected, 28 * 28은 512개의 노드
        self.fc2 = nn.Linear(512, 256)     # 512개의 노드를 256개로
        self.fc3 = nn.Linear(256, 10)      # 최종 아웃풋은 10개의 레이블

    def forward(self, x):                  # 순전파 feed forward
        x = x.view(-1, 28 * 28)            # 2차원을 1차원으로, 이를 Flatten 이라함
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)      # 최종 활성함수는 softmax인데, log.softmax
        return x                          # log.softmax는 좀 더 원활하게 계산

In [8]:
''' 7. Optimizer, Objective Function 설정하기 '''
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
criterion = nn.CrossEntropyLoss()

print(model)         

# optim 최적화, 즉 학습관련 기술은 SGD, momentum, Adagrad 등 있음
# 나중에 다시 한번 공부하도록 함.

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [9]:
''' 8. MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()                  # 최적화 초기화 
        output = model(image)                  # image 데이터를 통해 output
        loss = criterion(output, label)        # output과 label 비교가 loss
        loss.backward()                        # 역전파
        optimizer.step()

        if batch_idx % log_interval == 0:     # 모니터링하는 구간
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [10]:
''' 9. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()           # 평가상태
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]  # 가장 큰 값의 index
            correct += prediction.eq(label.view_as(prediction)).sum().item() # 정답갯수
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)      # 배치사이즈 별 loss
    test_accuracy = 100. * correct / len(test_loader.dataset) # 정확도 계산
    return test_loss, test_accuracy

In [11]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200) 
    # MLP모델, 학습데이터, optimizer - SGD
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

C:\Users\tjdqj\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.296040
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 2.323861
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 2.277985
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 2.325269
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 2.276271
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 2.277673
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 2.275075
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 2.257281
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 2.210144
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 2.277301

[EPOCH: 1], 	Test Loss: 2.2278, 	Test Accuracy: 47.52 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 2.225406
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 2.218935
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 2.162844
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 2.087265
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 1.935681
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 1.903102
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 1.625449
T